<a href="https://colab.research.google.com/github/Juuuuuuuuun/RL-Term-Project/blob/main/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 📌 1. 라이브러리 설치 및 환경 설정
# ============================================
# - transformers: BERT 임베딩 추출용
# - datasets: HF datasets (본 프로젝트에서는 사용 X, but RL 프로젝트 기본 세팅)
# - torch: PyTorch (모델 학습)
# - gymnasium: 강화학습 환경 구현
# ============================================

!pip install transformers datasets torch gymnasium==0.29.1

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

# ============================================
# 📌 2. GPU 사용 여부 자동 확인
# ============================================
# Colab에서 GPU가 켜져 있으면 모델/BERT forward 속도 매우 빠르게 수행 가능.
# ============================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.2.2
    Uninstalling gymnasium-1.2.2:
      Successfully uninstalled gymnasium-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.1 which is incompatible.


device(type='cpu')

In [ ]:
# ============================================
# 📌 3. 데이터 로드
# ============================================
# - encoding='utf-8' 또는 cp949 필요 (한국어 CSV 파일은 cp949인 경우 많음)
# ============================================

df = pd.read_csv('/content/df.csv', encoding='utf-8')

# ============================================
# 📌 4. 컬럼명 표준화
# ============================================
# GOODS_NAME → goods
# GOODS_OPTN_VLU → option
# MODEL_NAME → answer
# RL 환경이 읽기 쉽게 컬럼 이름을 통일함.
# ============================================

df = df.rename(columns={
    "GOODS_NAME": "goods",
    "GOODS_OPTN_VLU": "option",
    "MODEL_NAME": "answer"
})

df["goods"] = df["goods"].fillna("").astype(str)
df["option"] = df["option"].fillna("").astype(str)

# labels: 정답 후보 리스트 → 현재는 정답 1개로 구성
df["labels"] = df["answer"].apply(lambda x: [x])

In [ ]:
# ============================================
# 📌 5. BERT 모델 로드 (DistilBERT)
# ============================================
# - 텍스트에서 의미 임베딩(768차원)을 추출
# - 이후 강화학습에서는 이 임베딩만 사용하여 속도 대폭 향상
# - 모델은 GPU로 이동 (.to(device))
# ============================================

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [ ]:
# ============================================
# 📌 6. 텍스트를 DistilBERT 임베딩으로 변환하는 함수
# ============================================
# input: "상품명 + 옵션" 문자열
# output: 768차원 CLS vector
# ============================================

@torch.no_grad()
def embed_text(text):
    tokens = tokenizer(text, return_tensors="pt",
                       truncation=True, padding=False).to(device)
    output = model(**tokens).last_hidden_state[:, 0, :]  # CLS 토큰
    return output.squeeze().cpu().numpy()

# ============================================
# 📌 7. 전체 데이터에 대해 사전 임베딩 생성 (중요)
# ============================================
# - RL 환경 내에서 BERT를 매번 실행하면 매우 느림
# - 따라서 모든 데이터를 미리 embedding하여 메모리에 저장
# ============================================

emb_list = []

for text in tqdm(df["goods"] + " " + df["option"], total=len(df)):
    emb = embed_text(text)
    emb_list.append(emb)

df["embedding"] = emb_list

  0%|          | 3621/1048575 [08:16<34:18:21,  8.46it/s]

In [ ]:
https://colab.research.google.com/drive/1MlPSvM-RmXBPC6ICIcxSm6PcbxIX3gS6?usp=drive_link

In [ ]:
# ============================================
# 📌 8. 강화학습 환경 정의 (Gym Environment)
# ============================================
# - State: 사전 계산된 BERT 임베딩 (768차원)
# - Action: max_candidates 크기 (기본 10개)
# - Reward: 정답 선택 시 +1, 오답 선택 시 -1
# - Episode는 Single-step 구조 (문장 하나를 보고 바로 선택)
# ============================================

class ModelExtractEnv(gym.Env):
    def __init__(self, df, max_candidates=10):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.n = len(df)
        self.max_candidates = max_candidates

        # RL action space (정답 후보 중 하나 선택)
        self.action_space = spaces.Discrete(max_candidates)

        # observation_space = BERT embedding 768차원
        self.observation_space = spaces.Box(
            low=-5, high=5, shape=(768,), dtype=np.float32
        )

        self.index = 0

    def reset(self, seed=None, options=None):
        # 랜덤하게 하나의 상품 선택
        self.index = np.random.randint(0, self.n)
        row = self.df.iloc[self.index]

        # 바로 사전 계산된 embedding 사용 → 매우 빠름
        obs = row["embedding"]

        return obs, {}

    def step(self, action):
        row = self.df.iloc[self.index]
        labels = row["labels"]
        answer = row["answer"]

        # 정답 판단
        reward = 1.0 if action < len(labels) and labels[action] == answer else -1.0

        terminated = True  # Single-step MDP

        # 다음 상태는 의미 없음 → zero vector 반환
        return np.zeros((768,), dtype=np.float32), reward, terminated, False, {}

In [ ]:
# ============================================
# 📌 9. Actor-Critic 기반 모델 정의
# ============================================
# - PPO, A2C에서 공용으로 사용되는 뉴럴넷
# - Actor: 정책 π(a|s)
# - Critic: 상태 가치 함수 V(s)
# ============================================

class ActorCritic(nn.Module):
    def __init__(self, obs_dim, action_dim):
        super().__init__()

        # 정책 네트워크 (actor)
        self.actor = nn.Sequential(
            nn.Linear(obs_dim, 256), nn.ReLU(),
            nn.Linear(256, action_dim), nn.Softmax(dim=-1)
        )

        # 가치 네트워크 (critic)
        self.critic = nn.Sequential(
            nn.Linear(obs_dim, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.actor(x), self.critic(x)

# ============================================
# 📌 10. PPO Agent 정의
# ============================================
# - clipped objective 사용
# - 안정적인 policy 업데이트
# ============================================

class PPOAgent:
    def __init__(self, obs_dim, action_dim, lr=3e-4, gamma=0.99, eps_clip=0.2):
        self.policy = ActorCritic(obs_dim, action_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        self.eps_clip = eps_clip

    # 행동 선택
    def act(self, obs):
        obs = torch.tensor(obs, dtype=torch.float32)
        probs, _ = self.policy(obs)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item(), dist.log_prob(action).detach()

    # PPO 업데이트
    def update(self, memory):
        obs = torch.stack(memory["obs"]).detach()
        actions = torch.tensor(memory["actions"])
        rewards = torch.tensor(memory["rewards"]).detach()
        logprobs_old = torch.stack(memory["logprobs"]).detach()

        # Return 계산
        returns = []
        R = 0
        for r in reversed(rewards.tolist()):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)

        for _ in range(5):  # 여러 epoch 반복
            probs, values = self.policy(obs)
            dist = Categorical(probs)
            logprobs_new = dist.log_prob(actions)

            ratio = torch.exp(logprobs_new - logprobs_old)
            advantage = (returns - values.squeeze()).detach()

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-self.eps_clip, 1+self.eps_clip)*advantage

            loss = -torch.min(surr1, surr2).mean() + 0.5*(advantage**2).mean()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

# ============================================
# 📌 11. A2C Agent 정의
# ============================================
# - Advantage Actor-Critic
# - Baseline(value)을 활용하여 variance 감소
# ============================================

class A2CAgent:
    def __init__(self, obs_dim, action_dim, lr=1e-3, gamma=0.99):
        self.model = ActorCritic(obs_dim, action_dim)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma

    def act(self, obs):
        obs = torch.tensor(obs, dtype=torch.float32)
        probs, _ = self.model(obs)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item(), dist.log_prob(action)

    # A2C 업데이트
    def update(self, obs, action, reward, logprob):
        obs = torch.tensor(obs, dtype=torch.float32)
        probs, value = self.model(obs)

        advantage = reward - value

        actor_loss = -logprob * advantage
        critic_loss = advantage.pow(2)

        loss = actor_loss + critic_loss

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# ============================================
# 📌 12. Multi-Armed Bandit Agent 정의
# ============================================
# - 강화학습 중 가장 단순한 epsilon-greedy 탐색 기법
# - 비교 실험용 baseline으로 사용
# ============================================

class BanditAgent:
    def __init__(self, n_actions=10, eps=0.1):
        self.n_actions = n_actions
        self.eps = eps
        self.values = np.zeros(n_actions)
        self.counts = np.zeros(n_actions)

    def act(self):
        if np.random.rand() < self.eps:
            return np.random.randint(self.n_actions)
        return np.argmax(self.values)

    def update(self, action, reward):
        self.counts[action] += 1
        lr = 1 / self.counts[action]
        self.values[action] += lr * (reward - self.values[action])

In [ ]:
# ============================================
# 📌 13. 세 가지 알고리즘 학습 함수 정의
# ============================================

def train_PPO(env, episodes=20):
    agent = PPOAgent(768, env.max_candidates)
    for ep in range(episodes):
        obs, _ = env.reset()

        memory = {"obs": [], "actions": [], "rewards": [], "logprobs": []}

        action, logprob = agent.act(obs)
        _, reward, _, _, _ = env.step(action)

        memory["obs"].append(torch.tensor(obs))
        memory["actions"].append(action)
        memory["rewards"].append(reward)
        memory["logprobs"].append(logprob)

        agent.update(memory)

    return agent


def train_A2C(env, episodes=20):
    agent = A2CAgent(768, env.max_candidates)
    for ep in range(episodes):
        obs, _ = env.reset()
        action, logprob = agent.act(obs)
        _, reward, _, _, _ = env.step(action)
        agent.update(obs, action, reward, logprob)
    return agent


def train_Bandit(env, episodes=20):
    agent = BanditAgent(env.max_candidates)
    for ep in range(episodes):
        obs, _ = env.reset()
        action = agent.act()
        _, reward, _, _, _ = env.step(action)
        agent.update(action, reward)
    return agent

In [ ]:
# ============================================
# 📌 14. 평가 함수 정의
# ============================================
# - test_df를 사용하여 각 알고리즘별 accuracy 계산
# ============================================

def evaluate_agent(agent, df, mode="PPO"):
    correct = 0
    env = ModelExtractEnv(df)

    for i in range(len(df)):
        obs, _ = env.reset()

        if mode == "PPO":
            action, _ = agent.act(obs)
        elif mode == "A2C":
            action, _ = agent.act(obs)
        else:  # Bandit
            action = agent.act()

        row = df.iloc[env.index]
        labels = row["labels"]
        answer = row["answer"]

        if action < len(labels) and labels[action] == answer:
            correct += 1

    return correct / len(df)

In [ ]:
# ============================================
# 📌 15. Train/Test Split
# ============================================

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df, test_size=0.2, shuffle=True, random_state=42
)

train_env = ModelExtractEnv(df)

In [ ]:
# ============================================
# 📌 16. 세 강화학습 알고리즘 학습
# ============================================

ppo_agent = train_PPO(train_env)
a2c_agent = train_A2C(train_env)
bandit_agent = train_Bandit(train_env)

# ============================================
# 📌 17. Test Accuracy 비교 출력
# ============================================

ppo_acc = evaluate_agent(ppo_agent, test_df, mode="PPO")
a2c_acc = evaluate_agent(a2c_agent, test_df, mode="A2C")
bandit_acc = evaluate_agent(bandit_agent, test_df, mode="Bandit")

print("=== Test Accuracy ===")
print(f"PPO:    {ppo_acc:.2f}")
print(f"A2C:    {a2c_acc:.2f}")
print(f"Bandit: {bandit_acc:.2f}")